## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [10]:
# Set the file path to import the WeatherPy_database.csv file
file_path = "../Weather_Database/WeatherPy_database.csv"

# Load the CSV file into a Pandas DataFrame
city_data_df = pd.read_csv("Weatherpy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Ambovombe,-25.1667,46.0833,68.52,88,2,4.05,MG,2022-11-17 01:29:25
1,1,Mataura,-46.1927,168.8643,58.48,85,100,2.80,NZ,2022-11-17 01:29:25
2,2,Jamestown,42.0970,-79.2353,34.02,70,100,17.00,US,2022-11-17 01:29:26
3,3,Bethel,41.3712,-73.4140,42.76,75,0,5.75,US,2022-11-17 01:25:33
4,4,Punta Arenas,-53.1500,-70.9167,50.11,66,0,6.91,CL,2022-11-17 01:29:26


In [11]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 45
What is the maximum temperature you would like for your trip? 65


In [12]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,Mataura,-46.1927,168.8643,58.48,85,100,2.80,NZ,2022-11-17 01:29:25
4,4,Punta Arenas,-53.1500,-70.9167,50.11,66,0,6.91,CL,2022-11-17 01:29:26
6,6,Hobart,-42.8794,147.3294,60.06,70,75,13.80,AU,2022-11-17 01:29:27
7,7,Cape Town,-33.9258,18.4232,61.27,71,0,13.80,ZA,2022-11-17 01:24:54
8,8,Bluff,-46.6000,168.3333,58.80,87,100,12.66,NZ,2022-11-17 01:29:27
10,10,Kamaishi,39.2667,141.8833,47.84,54,42,5.53,JP,2022-11-17 01:29:28
11,11,Busselton,-33.6500,115.3333,59.90,70,98,19.35,AU,2022-11-17 01:29:23
19,19,Ushuaia,-54.8000,-68.3000,51.46,54,75,4.61,AR,2022-11-17 01:29:30
25,25,Pisco,-13.7000,-76.2167,64.45,82,0,14.97,PE,2022-11-17 01:25:16
26,26,Coquimbo,-29.9533,-71.3436,58.73,94,0,6.91,CL,2022-11-17 01:29:32


In [13]:
# 4a. Determine if there are any empty rows.
print(preferred_cities_df.count())

City_ID       173
City          173
Lat           173
Lng           173
Max Temp      173
Humidity      173
Cloudiness    173
Wind Speed    173
Country       171
Date          173
dtype: int64


In [14]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()

In [23]:
import numpy as np
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = np.nan

In [24]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json" 

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [26]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()

In [27]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [31]:
clean_hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Mataura,NZ,58.48,-46.1927,168.8643,Ellie's Villa
4,Punta Arenas,CL,50.11,-53.1500,-70.9167,Hotel Hain
6,Hobart,AU,60.06,-42.8794,147.3294,St Ives Hobart Accommodation Tasmania
7,Cape Town,ZA,61.27,-33.9258,18.4232,"Radisson Blu Hotel Waterfront, Cape Town"
8,Bluff,NZ,58.80,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park


In [32]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd> and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [34]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))

Figure(layout=FigureLayout(height='420px'))